<center>
<h1>Tensorflow implementation of unsupervised single image depth prediction using a convolutional neural network</h1>
    
<h3>based on <i>"Unsupervised Monocular Depth Estimation with Left-Right Consistensy"</i><br> Clément Godard, Oisin Mac Aodha and Gabriel J. Brostow.</h3>
</center>


<hr>
<h2> <u>I Approach</u> </h2>

Godard et al. propose a training objective that enables a convolutional neural network to learn to perform single image depth estimation, despite the absence of ground truth depth data. Ex-ploiting epipolar geometry constraints, disparity
images are generated by training the network with an image reconstruction loss.

At training time, the CNN has access to two images namely the left and right color images from a calibrated stereo pair, captured at the same moment in time. Instead of trying to directly predict the depth, the CNN attempt to find the dense correspondence field that would enable it to reconstruct the right image, when applied to the left image. Similarly, the CNN can also estimate the left image given the right one.

naïvely learning to generate the right image (target) by sampling from the left one (input) will produce disparities aligned with the right image. This is illustrated in the figure below:<br>
<img title="figure1" src="figures/figure1.png"><br>

However, we want the output disparity map to align with the input left image, meaning the network has to sample from the
right image. We could instead train the network to generate the left view by sampling from the right image, thus creating a left view aligned disparity map (No LR baseline):<br>
<img title="figure2" src="figures/figure2.png"><br>

While this alone works, the inferred disparities exhibit ‘texture-copy’ artifacts and errors at depth discontinuities. This is illustrated in the two depth maps below, where the left one corresponds to No LR baseline and the right one the to the combined constructions left to right and right to left:<br>
<img style="float:left; display: inline" height="200" width="400" title="figure4.1" src="figures/figure4.1.png">
<img style="both: clear" height="200" width="400" title="figure4.0" src="figures/figure4.0.png"><br>

This issue is solved by training the network to predict the disparity maps for both views by sampling from the opposite input images. This still only requires a single left image as input to the convolutional layers and the right image is only used during training:<br>
<img title="figure3" src="figures/figure3.png"><br>

Enforcing consistency between both disparity maps using this
left-right consistency cost leads to more accurate results.

<hr>

<h2> <u>II Implementation</u> </h2>



The simple mode monodepth_simple.py provides a shorthand to infer with the model on a test image, which allows us to quickly visualize the results of the implementation. The command below requires the image path and the trained model. We take two different images (000080.png from KITTI and berlin.jpg from Cityscapes) and test them on both of the trained cityscapes and kitti models with resnet50 as encoder.

Input 000080.png from KITTI:
<img src="000080.png">
Output 000080_disp_cityscapes.png:
<img src="000080_disp_cityscapes.png">
<br>
<hr>
<br>
Input berlin.jpg from Cityscapes:
<img src="berlin.jpg" height="450" width="800">
Output berlin_disp_cityscapes.png:
<img src="berlin_disp_cityscapes.png" height="450" width="800">

In [ ]:
python monodepth_simple.py --image_path PATH_TO_IMAGE --checkpoint_path ./trained_models/model_kitti

Input 000080.png from KITTI:
<img src="000080.png">
Output 000080_disp_cityscapes.png:
<img src="000080_disp_kitti.png">
<br>
<hr>
<br>
Input berlin.jpg from Cityscapes:
<img src="berlin.jpg" height="450" width="800">
Output berlin_disp_cityscapes.png:
<img src="berlin_disp_kitti.png" height="450" width="800">